In [ ]:
!pip install sagemaker==1.72.0

In [59]:
import os
import boto3
import re
import sagemaker
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import sagemaker.amazon.common as smac
import scipy
from sklearn.model_selection import train_test_split
from sagemaker.amazon.amazon_estimator import get_image_uri

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
session = sagemaker.Session()
bucket = session.default_bucket()

In [60]:
X_sparse = scipy.sparse.load_npz('./readmissions_X.npz')
y_sparse = scipy.sparse.load_npz('./readmissions_y.npz')
X = pd.DataFrame(np.array(X_sparse.todense()))
y = pd.DataFrame(np.array(y_sparse.todense()).reshape(-1,1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33)

data_dir = './data/'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
pd.DataFrame(X_test).to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)
pd.concat([y_val, X_val], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)
pd.concat([y_train, X_train], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

test_location = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

In [ ]:
prefix = 'readmission-xgboost'

from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner
container = get_image_uri(session.boto_region_name, 'xgboost')

xgb = sagemaker.estimator.Estimator(container, 
                                    role,                                   
                                    train_instance_count=1,                 
                                    train_instance_type='ml.m4.xlarge',    
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=session)

xgb.set_hyperparameters(silent=0,
                        objective='binary:logistic',
                        early_stopping_rounds=10,
                        num_round=500)

xgb_hyperparameter_tuner = HyperparameterTuner(estimator = xgb, 
                                               objective_metric_name = 'validation:auc', 
                                               objective_type = 'Maximize',
                                               max_jobs = 20,
                                               max_parallel_jobs = 3,
                                               hyperparameter_ranges = {
                                                    'max_depth': IntegerParameter(3, 12),
                                                    'eta'      : ContinuousParameter(0.05, 0.5),
                                                    'min_child_weight': IntegerParameter(2, 8),
                                                    'subsample': ContinuousParameter(0.5, 0.9),
                                                    'gamma': ContinuousParameter(0, 10)})

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

xgb_hyperparameter_tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

xgb_hyperparameter_tuner.wait()

In [ ]:
my_tuner_analytics = xgb_hyperparameter_tuner.analytics()
my_dataframe = my_tuner_analytics.dataframe()

best_job = xgb_hyperparameter_tuner.best_training_job()
my_dataframe[my_dataframe['TrainingJobName'] == best_job]

In [ ]:
xgb_best = sagemaker.estimator.Estimator.attach(xgb_hyperparameter_tuner.best_training_job())
xgb_transformer = xgb_best.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')
xgb_transformer.transform(test_location, content_type='text/csv', split_type='Line')

xgb_transformer.wait()

In [ ]:
!aws s3 cp --recursive $xgb_transformer.output_path $data_dir

output = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
predictions = [round(num) for num in output.squeeze().values]
probs = [p for p in output.squeeze().values]

from sklearn.metrics import accuracy_score
print (accuracy_score(y_test, predictions))

from sklearn.metrics import roc_auc_score
print (roc_auc_score(y_test, probs))

'readmissions/linreg'

In [61]:
prefix = "readmissions/linreg"

import io
import numpy as np
import sagemaker.amazon.common as smac

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, X_train.values.astype("float32"), y_train.values[:,0].astype("float32"))
buf.seek(0)

key = "recordio-pb-data"
boto3.resource("s3").Bucket(bucket).Object(os.path.join(prefix, "train", key)).upload_fileobj(buf)
s3_train_data = f"s3://{bucket}/{prefix}/train/{key}"
print(f"uploaded training data location: {s3_train_data}")

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, X_val.values.astype("float32"), y_val.values[:,0].astype("float32"))
buf.seek(0)

key = "recordio-pb-data"
boto3.resource("s3").Bucket(bucket).Object(os.path.join(prefix, "val", key)).upload_fileobj(buf)
s3_val_data = f"s3://{bucket}/{prefix}/val/{key}"
print(f"uploaded validation data location: {s3_val_data}")

uploaded training data location: s3://sagemaker-us-east-1-345989147144/readmissions/linreg/train/recordio-pb-data
uploaded validation data location: s3://sagemaker-us-east-1-345989147144/readmissions/linreg/val/recordio-pb-data


In [62]:
container = get_image_uri(session.boto_region_name, 'linear-learner')

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [63]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, CategoricalParameter, HyperparameterTuner
linear = sagemaker.estimator.Estimator(
         container,
         role,
         train_instance_count=1,
         train_instance_type="ml.c4.xlarge",
         output_path='s3://{}/{}/output'.format(session.default_bucket(), prefix),
         sagemaker_session=session)

linear.set_hyperparameters(feature_dim=105, predictor_type="binary_classifier")

lin_hyperparameter_tuner = HyperparameterTuner(estimator = linear, 
                                               objective_metric_name = 'validation:binary_classification_accuracy', 
                                               objective_type = 'Maximize',
                                               max_jobs = 20,
                                               max_parallel_jobs = 3,
                                               hyperparameter_ranges = {
                                                    'wd': ContinuousParameter(0.1, 1.0),
                                                    'l1'      : ContinuousParameter(0.1, 1.0),
                                                    'learning_rate': ContinuousParameter(0.1, 1.0),
                                                    'mini_batch_size': IntegerParameter(100, 5000),
                                                    'use_bias': CategoricalParameter([True, False])})

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [64]:
lin_hyperparameter_tuner.fit({'train': s3_train_data, 'validation': s3_val_data})
lin_hyperparameter_tuner.wait()

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


................................................................................................................................................................................................................................................................................................................................................................................!


In [65]:
my_tuner_analytics = lin_hyperparameter_tuner.analytics()
my_dataframe = my_tuner_analytics.dataframe()

best_job = lin_hyperparameter_tuner.best_training_job()
my_dataframe[my_dataframe['TrainingJobName'] == best_job]

,l1,learning_rate,mini_batch_size,use_bias,wd,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
15,1.0,0.285439,2043.0,False,0.873948,linear-learner-211230-1808-005-a9e8df26,Completed,0.603661,2021-12-30 18:16:02+00:00,2021-12-30 18:17:25+00:00,83.0


In [66]:
lin_best = sagemaker.estimator.Estimator.attach(lin_hyperparameter_tuner.best_training_job())
lin_transformer = lin_best.transformer(instance_count = 1, instance_type = 'ml.m4.xlarge')
lin_transformer.transform(test_location, content_type='text/csv', split_type='Line')

lin_transformer.wait()

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


2021-12-30 18:17:25 Starting - Preparing the instances for training
2021-12-30 18:17:25 Downloading - Downloading input data
2021-12-30 18:17:25 Training - Training image download completed. Training in progress.
2021-12-30 18:17:25 Uploading - Uploading generated training model
2021-12-30 18:17:25 Completed - Training job completedDocker entrypoint called with argument(s): train
Running default environment configuration script
[12/30/2021 18:17:11 INFO 140579261622080] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss': 'auto', 'margin':

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


...................................Docker entrypoint called with argument(s): serve
Running default environment configuration script
Docker entrypoint called with argument(s): serve
Running default environment configuration script
[12/30/2021 18:45:36 INFO 140044959930176] loaded entry point class algorithm.serve.server_config:config_api
[12/30/2021 18:45:36 INFO 140044959930176] loading entry points
[12/30/2021 18:45:36 INFO 140044959930176] loaded request iterator application/json
[12/30/2021 18:45:36 INFO 140044959930176] loaded request iterator application/jsonlines
[12/30/2021 18:45:36 INFO 140044959930176] loaded entry point class algorithm.serve.server_config:config_api
[12/30/2021 18:45:36 INFO 140044959930176] loading entry points
[12/30/2021 18:45:36 INFO 140044959930176] loaded request iterator application/json
[12/30/2021 18:45:36 INFO 140044959930176] loaded request iterator application/jsonlines
[12/30/2021 18:45:36 INFO 140044959930176] loaded request iterator applicatio

In [67]:
!aws s3 cp --recursive $lin_transformer.output_path $data_dir

results = pd.read_csv(os.path.join(data_dir, 'test.csv.out'), header=None)
predictions = [int(p.split(':')[1]) for p in results[0]]
probs = [float(p.split(':')[1].split('}')[0]) for p in results[1]]

download: s3://sagemaker-us-east-1-345989147144/linear-learner-211230-1808-005-a9e8df26-2021-12-30-18-39-54-907/test.csv.out to data/test.csv.out


In [68]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.6007314152561571

In [69]:
from sklearn.metrics import roc_auc_score
print (roc_auc_score(y_test, probs))

0.6364523514048408


In [72]:
lin_hyperparameter_tuner.best_training_job()

'linear-learner-211230-1808-005-a9e8df26'

In [39]:
session.default_bucket()

'sagemaker-us-east-1-345989147144'

In [77]:
prefix

'readmissions/linreg'

In [79]:
import os
import mxnet as mx
import boto3

key = '/readmissions/linreg/output/linear-learner-211230-1808-005-a9e8df26/output/'
boto3.resource('s3').Bucket(session.default_bucket()).download_file(key, 'model.tar.gz')
 
os.system('tar -zxvf model.tar.gz')
 
# Linear learner model is itself a zip file, containing a mxnet model and other metadata.
# First unzip the model.
os.system('unzip model_algo-1') 
 
# Load the mxnet module
mod = mx.module.Module.load("mx-mod", 0)
 
# model's weights
mod._arg_params['fc0_weight'].asnumpy().flatten()
 
# model bias
mod._arg_params['fc0_bias'].asnumpy().flatten()
 
# Using the model for prediction
# First create a mxnet data iterator:
# https://mxnet.incubator.apache.org/tutorials/basic/data.html#reading-data-in-memory
# https://mxnet.incubator.apache.org/tutorials/basic/data.html#reading-data-from-csv-files
data_iter = create_data_iter()
 
# Next bind the module with the data shapes.
mod.bind(data_shapes=data_iter.provide_data)
 
# Predict
mod.predict(data_iter)

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found